In [1]:
from glob import glob
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.cross_validation import StratifiedShuffleSplit
import itertools
from collections import defaultdict

all_model_scores = defaultdict(list)

# For every data set in the data repository...
for dataset in glob('data/ecoli.csv.gz'):
    # Read the data set into memory
    input_data = pd.read_csv(dataset, compression='gzip')
    
    for dataset_repeat in range(1, 31):
        # Divide the data set into a training and testing sets, each time with a different RNG seed
        training_indices, testing_indices = next(iter(StratifiedShuffleSplit(input_data['class'].values,
                                                                             n_iter=1,
                                                                             train_size=0.75,
                                                                             test_size=0.25,
                                                                             random_state=dataset_repeat)))
        
        training_features = input_data.loc[training_indices].drop('class', axis=1).values
        training_classes = input_data.loc[training_indices, 'class'].values

        testing_features = input_data.loc[testing_indices].drop('class', axis=1).values
        testing_classes = input_data.loc[testing_indices, 'class'].values
        
        # These are parameters listed in the project document
        C_list = [0.01, 0.1, 0.5, 1.0, 10.0, 50.0, 100.0]
        loss_list = ['hinge', 'squared_hinge']
        penalty_list = ['l2']
        max_iter_list = [10, 100, 500, 1000, 2000]
        # For every DecisionTreeClassifier parameter combination...
        for (C, loss, penalty, max_iter) in itertools.product(C_list, loss_list, penalty_list, max_iter_list):
            # Create and fit the model on the training data
            clf = LinearSVC(C = C, loss = loss, penalty = penalty, max_iter = max_iter)
            clf.fit(training_features, training_classes)
            
            # Store the model's score with the key (name of data set, model, model parameters)
            all_model_scores[(dataset,
                              'LinearSVC',
                              (C, loss, penalty, max_iter))].append(clf.score(testing_features,
                                                                                      testing_classes))

In [2]:
import os
import gzip

if not os.path.isdir('benchmark_results'):
    os.mkdir('benchmark_results')

with gzip.open('benchmark_results/LinearSVC_eoli-benchmarks.tsv.gz', 'wb') as out_file:
    header_text = '\t'.join(['dataset',
                              'model',
                              'parameters',
                              'testing_score']) + '\n'
    out_file.write(header_text.encode('UTF-8'))
    
    for (dataset, model, params) in all_model_scores:
        param_string = ''
        param_string += 'C={},'.format(params[0])
        param_string += 'loss={},'.format(params[1])
        param_string += 'penalty={},'.format(params[1])
        param_string += 'max_iter={},'.format(params[1])
        
        for testing_score in all_model_scores[(dataset, model, params)]:
            out_text = '\t'.join([dataset[5:-7],
                                  model,
                                  param_string,
                                  str(testing_score)]) + '\n'
            out_file.write(out_text.encode('UTF-8'))

In [4]:
import pandas as pd

pd.read_csv('benchmark_results/LinearSVC_eoli-benchmarks.tsv.gz', compression='gzip', sep='\t')

,dataset,model,parameters,testing_score
0,ecoli,LinearSVC,"C=0.01,loss=hinge,penalty=hinge,max_iter=hinge,",0.475610
1,ecoli,LinearSVC,"C=0.01,loss=hinge,penalty=hinge,max_iter=hinge,",0.463415
2,ecoli,LinearSVC,"C=0.01,loss=hinge,penalty=hinge,max_iter=hinge,",0.475610
3,ecoli,LinearSVC,"C=0.01,loss=hinge,penalty=hinge,max_iter=hinge,",0.487805
4,ecoli,LinearSVC,"C=0.01,loss=hinge,penalty=hinge,max_iter=hinge,",0.475610
5,ecoli,LinearSVC,"C=0.01,loss=hinge,penalty=hinge,max_iter=hinge,",0.463415
6,ecoli,LinearSVC,"C=0.01,loss=hinge,penalty=hinge,max_iter=hinge,",0.487805
7,ecoli,LinearSVC,"C=0.01,loss=hinge,penalty=hinge,max_iter=hinge,",0.439024
8,ecoli,LinearSVC,"C=0.01,loss=hinge,penalty=hinge,max_iter=hinge,",0.463415
9,ecoli,LinearSVC,"C=0.01,loss=hinge,penalty=hinge,max_iter=hinge,",0.500000
